# CAO Points Analysis
## Author: Ross Downey

http://www2.cao.ie/points/l8.php
***


In [1]:
# HTTP Requests Python Module
import requests as rq

# Regular expressions module
import re

# Dates and time module
import datetime as dt

In [2]:
# Retrieving the 2021 CAO from this URL using the requests module
resp = rq.get('http://www2.cao.ie/points/l8.php')

# Veryifing Python has downloaded the data from the URL, [200] means it's ok
resp

<Response [200]>

<br>  

## Ensuring original dataset is saved
***

In [3]:
# Saving current date and time as a function
now = dt.datetime.now()

# Convert date and time to a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [4]:
# Filepath to be created in data folder to save original data
filepath = 'data/cao2021_' + nowstr + '.html'

<br>

## Error on Server
***
Issue with the character set defined on the CAO server.  
The server says we should decode as per:  
    Content-Type: text/html; charset=iso-8859-1  
On decoding as per 'iso-8859-1' it was noted that one line is using \x96,  
which is not defined in iso-8859-1.  
The similar decoding standard 'cp1252' will be used, which has a codepoint for \x96.

In [5]:
# Amending the original encoding
original_encoding = resp.encoding

# Changing to 'cp1252'
resp.encoding = 'cp1252'

In [6]:
# Saving the original file downloaded from server
with open(filepath, 'w') as f:
    f.write(resp.text)

In [7]:
# Compiling the regular expression for the matching lines.
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')
# This gives us only the lines in the original HTML code that have CAO points

<br>

## Explanation of Regular Expression
***
re.compile 
r = converting html flags to raw strings
[A-Z]{2} = Everything with 2 letters
[0-9]{3} = Followed by 3 numbers, this is each course code e.g. AL801

Two spaces followed by (.*) is space then wildcard, this is the course title string

[0-9]{3} This is the number of points required

(\*?) is a wildcard search, but some of these are actual asterisks in the html so using question mark

and asterisk at the end also as a wildcard



In [15]:
# Creating filepath to save the new csv file
filepath = 'data/cao2021_csv_' + nowstr + '.csv'


# Adding a counter to total the number of courses processed
no_lines = 0

# Opening filepath to save new csv file
with open(filepath, 'w') as f:
    
# Looping through the lines of the data 
    for line in resp.iter_lines():
        
        # Decode the line but using the 'wrong' encoding
        dline = line.decode('cp1252')
    
        # Using regular expression to match lines with courses and points only
        if re_course.fullmatch(dline):
        
            # Adding 1 for each line matched to give an overall total using the counter
            no_lines = no_lines + 1
        
            # Split the line based on number of spaces (i.e. two spaces or more) using '  +'
            linesplit = re.split('  +', dline)
        
            # Rejoin the substrings after splitting but with commas in between
            # This removes all of the spaces in the original data
            # Rejoined and using '\n' to add a new line for each course
            f.write(','.join(linesplit) + '\n')
        
# Displaying the total number of lines processed
print(f"Total number of lines is {no_lines}. ")

Total number of lines is 922. 


<br>

## Cleaning up csv file to remove special characters  
Need to remove any special characters ( '#', '@' etc.) from the data to peform analysis on the 

In [ ]:
string = open('cao2021_csv_20211019_213048.csv').read()
new_str = re.sub('[^a-zA-Z0-9\n\.]', ' ', string)
open('cao2021_cleaned.txt', 'w').write(new_str)

***
## End